<a href="https://colab.research.google.com/github/jainstuti/Tweets_classification/blob/main/dt_TMLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [36]:
#importing dataset
dataset = pd.read_csv('Twitter_Data.csv')

In [37]:
dataset=dataset.head(30000)

In [38]:
dataset.shape

(30000, 2)

In [39]:
# to get the values in categorical data and their counts in the data
dataset['category'].value_counts()

 1.0    12718
 0.0    10232
-1.0     7050
Name: category, dtype: int64

In [40]:
dataset.describe()

,category
count,30000.000000
mean,0.188933
std,0.789467
min,-1.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [41]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   clean_text  29999 non-null  object 
 1   category    30000 non-null  float64
dtypes: float64(1), object(1)
memory usage: 468.9+ KB


In [42]:
dataset.isnull().sum()


clean_text    1
category      0
dtype: int64

In [43]:
is_NaN = dataset.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dataset[row_has_NaN]

print(rows_with_NaN)

    clean_text  category
148        NaN       0.0


In [44]:
dataset=dataset.dropna()

In [45]:
dataset = dataset.reset_index()

In [46]:
dataset.shape

(29999, 3)

In [47]:
dataset.head()

,index,clean_text,category
0,0,when modi promised “minimum government maximum...,-1.0
1,1,talk all the nonsense and continue all the dra...,0.0
2,2,what did just say vote for modi welcome bjp t...,1.0
3,3,asking his supporters prefix chowkidar their n...,1.0
4,4,answer who among these the most powerful world...,1.0


In [48]:
dataset =dataset.drop(['index'], axis = 1)

In [49]:
#cleaning the text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 29999):
  tweet = re.sub("[^a-zA-Z]", " ", dataset['clean_text'][i])
  tweet = tweet.lower()
  tweet = tweet.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  tweet = [ps.stem(word) for word in tweet if not word in set(all_stopwords)]
  tweet = ' '.join(tweet)
  corpus.append(tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
#creating bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

In [51]:
X.shape

(29999, 3000)

In [52]:
#slpiiting data into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [54]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [55]:
y_pred = classifier.predict(X_test)

In [56]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 913  177  294]
 [ 158 1662  199]
 [ 330  282 1985]]


0.76

In [57]:
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)
print(np.mean(recall))
print(np.mean(precision))

0.7490684487131444
0.745439750512063


In [34]:
f1Score = 2 * precision * recall / (precision + recall)
print(np.mean(f1Score))

0.7418970334140856


In [59]:
print(precision)

[0.65167737 0.78359264 0.80104923]
